# FIT5148 - Distributed Databases and Big Data

# Assignment 1 - Solution Workbook

**Your details**
- Name: Hamidreza Mohammadiandehkordi
- Student ID: 28855205

- Name: Sunil Cyriac
- Student ID: 29003164

Let's get started!

# Loading the FireData.csv and ClimateData.csv
- In this part we load both csv files into the main memory for further queries.
- We used CSV module to implement, reading data in CSV format.
- A " row_id " also added to dataset for saving the position of original CSV file.

**Instruction**
- If the csv file is not in the same directory of the python notebook file then a complete directory of the file with the name of the file should be provided.
- If the csv file is not in the same directory of the python notebook file then type in the name of the file

In [17]:
import csv
from multiprocessing import Pool
import multiprocessing as mp
import sys

# Load the first CSV file  as "ClimateData" and "FireData"
def import_csv(csvFile):
    """Import CSV file to add the records to the relevant variable.

    Args:
        csvFile -- Location and the name of the csvFile( i.e C:/FireData.csv or FireData.csv which is
        located in the same folder as the *.ipynb file)

    Returns:
        data -- loaded data ready for processing

    """
    # Empty List to add the records of csvfile to it
    data = []
    # Read the file rows and add them to data list
    with open(csvFile, 'r') as fp:
        # To save position of each record in the original csvfile
        row_id = 0
        # Retunr a reader object which will iterate over lines in the given csvfile
        reader = csv.reader(fp, delimiter=',', quotechar='"',skipinitialspace=True)
        # Skip the headers
        next(reader, None)
        for row in reader:
            # Save the position of original file as a row_id
            row.append(str(row_id)) 
            row_id += 1
            data.append(row)
    return data

# Load the FireData.csv into FireData hence storing it into the memory
FireData = import_csv('FireData.csv')
# Load the ClimateData.csv into ClimateData hence storing it into the memory
ClimateData = import_csv('ClimateData.csv')

# Search Algorithms
**Description**

In this part all search algorithms that used in the assignment are defined.

In [18]:
# Serial Search - Binary Search Algorithm
def binary_search(data,index_number,key):
    """A search function for doing binary search on data based on key value

    Args:
        data: The data we want to search
        index_number: which item (column) in data records with index_number
        n_processor: number of processors

    Returns:
        result: the record found and its row_id

    """
    for line in range(len(data)):
        result = []
        # Lower bund of the list
        lower = 0
        # Middle of the list
        middle = 0
        # Upper bund of the list
        upper = len(data)
        
        # If the condition meet iterates over the data until the key has been found
        while (lower < upper):
            middle = int((lower + upper)/2)
            # Find the given key
            if data[middle][index_number] == key:
                return data[middle],data[middle][9]
            # Reduce the bupper bund if the key is smaller than query
            elif key < data[middle][index_number]:
                upper = middle -1
            # Increase the lowe bund if the key is bigger than query
            elif key > data[middle][index_number]:
                lower = middle +1
        # If there is no match add empty string to the list        
        result = [['' for i in range(9)],-1]
        return result    

In [19]:
# Serial Search - Linear Search Algorithm
def linear_search(data, query):
    """ Perform linear search on data for the given key
    Arguments:
        data -- an input dataset which is a list or a numpy array
        query -- an query record
    Return:
        result -- the record of searched query
    """
    result = []
    for item in data:
        if query == item[0]:
            result.append(item[1:])
    return result

In [20]:
# Continuous Range Search Algorithm
def range_search(data,index_number,query_range):
    """Fucntion to search in data for a specified range by index_number
    
        Arguments:
        Data -- sets of record need to be searched
        index_number -- index_number of query in the list or record
        query_range -- the range of query that we want to get the results for
        
    Return:
        result -- the matched record for the query range
    """
    result = []
    
    # For records in data check the the range of the query and specified item's value
    for x in data:
        val = int(x[index_number])
        val_hash = val
        if val >= query_range[0] and val <= query_range[1]:
            result_item = x[0:]
            result.append(result_item)
    return result

# Data Partitioning Methods
This part we define all the partitioning methods used for this assignment

In [21]:
# Hash Function
def s_hash(x ,index_number, n_processor):
    """A hash function to save the reminder of an item.

    Args:
        x: The value we want to hash.
        index_number:if the x is a list we specify which item with index_number
        n_processor: number of processors

    Returns:
        result: the hash we created from x (by taking the remainder)

    """
    # If x is list
    if type(x) is list:
        # Get the value from record 
        temp = x[index_number]
        # Replace the - characters
        filtered = temp.replace("-","")
        # Get the remainder as int
        result = int(filtered)%n_processor
    # If x is string
    elif type(x) is str:
        # Replace - characters
        filtered = x.replace("-","")
        # Get the remainder as int
        result = int(filtered)%n_processor
        
    return result

def h_partition(data,index_number ,n_processor):
    """a partitioning function based on hash partitioning

    Args:
        data: The data we want to partition
        index_number: which item (column) in data records with index_number
        n_processor: number of processors

    Returns:
        dic: a dictionary of partitioned data based on hash value

    """
    dic = {}
    for x in data:
        h = s_hash(x,index_number,n_processor)
        # If its not in our hashes, add it
        if (h in dic.keys()):
            s = dic[h] 
            s.append(x)
            dic[h] = s
        # Else add it to the same hash value list
        else:
            s = []
            s.append(x)
            dic[h] = s
    return dic


In [22]:
def h_partition_joined_tables(data,index_number ,n_processor):
    """a partitioning function based on hash partitioning

    Args:
        data: The data we want to partition
        index_number: which item (column) in data records with index_number
        n_processor: number of processors

    Returns:
        dic: a dictionary of partitioned data based on hash value

    """
    dic = {}
    for x in data:
        # Hasing base on hash function
        h = s_hash(x,index_number,n_processor)
        # If its not in our hashes, add it
        if (h in dic.keys()):
            s = dic[h] 
            s.append([x[13], x[2], x[17],x[index_number]])
            dic[h] = s
        # Else add it to the same hash value list
        else:
            s = []
            s.append([x[13], x[2], x[17],x[index_number]])
            dic[h] = s
    return dic

In [23]:
def hash_partition(data,index_number ,n_processor):
    """a partitioning function based on hash partitioning

    Args:
        data: The data we want to partition
        index_number: which item (column) in data records with index_number
        n_processor: number of processors

    Returns:
        dic: a dictionary of partitioned data based on hash value

    """
    dic = {}
    for x in data:
        # If its not in our hashes, add it
        h = s_hash(x,index_number,n_processor)
        if (h in dic.keys()):
            s = dic[h] 
            s.append([x[index_number], x[0], x[1], x[5],x[6],x[7],x[8]])
            dic[h] = s
        # Else add it to the same hash value list
        else:
            s = []
            s.append([x[index_number], x[0], x[1], x[5],x[6] ,x[7],x[8]])
            dic[h] = s
    return dic

In [24]:
def rr_partition(data,n):
    """Perform data partitioning on data
    Arguments:
        data -- an input dataset which is a list
        n -- the number of processors
    Return:
        result -- the paritioned subsets of D
    """
    result = []
    #for every bin create an empty list to add the partitioned to it
    for i in range(n):
        result.append([])
    # Calculate the number of the elements to be allocated to each bin
    n_bin = len(data)/n
    # For each bin, perform the following
    for index, element in enumerate(data):
        # Calculate the index of the bin that the current data point will be assigned
        index_bin = index%n
        result[index_bin].append(element)
    return result

# Task 1.1 (Parallel Search)
Write an algorithm to search climate data for the records on ​15th December 2017​. Justify
your choice of the data partition technique and search technique you have used

**Justification**
- For this task we are using hash partition as our partition method and binary search as our searching technique.
- This arrangment is best for exact matach retrieval based on the partitioning attributes, where the processors containing the desired record can be accessed directly provided that the hash function is based on the "date" attribute of ClimateData that is also the same attribute for the exact retrieval.
- In this case, only selected processing elements are activated as the hold the candidate records which is "2017-12-15" in this task, thereby reducing the total cost and also the reducing the boundary of the search.
- The reason we used binary search is because every time makes a comparison and fails to find the desired item, it eliminates half of the remaining list that must be search.
- In this case, the query is an exact match thus by using binary search the cost and time of searching is effectively reduced.


In [25]:
def parallel_search_exact(data,index_number,key,n_processor, m_partition,m_search):
    """ Perform parallel search for exact match on data for the given key
    Args:
        data -- an input dataset which is a list
        index_number -- index number of the key
        key -- a query record
        n_processor -- the number of parallel processors
        m_partition -- a data partitioning method
        m_search -- a search method
    Return:
        results -- the matched record information
    """
    results = []
    # Pool: a Python class enabling parallel processing.
    # set the number of processors for running at the same time
    pool = Pool(processes = n_processor) 
    # Store the partitioned list in DD
    # Each element in DD has a pair (hash key: records)
    DD = m_partition(data,index_number,n_processor)
    # hashing the key query
    query_hash = s_hash(key,index_number,n_processor)
    # Storing the partition bbin into d
    d = list(DD[query_hash])
    result = pool.apply(m_search,[d,index_number,key])
    pool.close()
    results.append(result)
    return results
task1 = parallel_search_exact(ClimateData,1,'2017-12-15',3,h_partition,binary_search)
print("Station: " + task1[0][0][0] +"\nDate: "+task1[0][0][1]+
      "\nAir Temprature: "+task1[0][0][2]+"\nRelative Humidity: "+task1[0][0][3]
      +"\nWind Speed(knots): "+ task1[0][0][4]+"\nMax WindSpeed: "+task1[0][0][5]
      +"\nMAX: "+ task1[0][0][6]+"\nMin: "+task1[0][0][7]
      +"\nPrecipitation: "+task1[0][0][8]+"\nPosition: "+ task1[0][0][9])

Station: 948702
Date: 2017-12-15
Air Temprature: 18
Relative Humidity: 52
Wind Speed(knots): 7.1
Max WindSpeed: 14
MAX: 74.5*
Min: 53.1
Precipitation: 0.00I
Position: 348


# Task 1.2 (Parallel Search)
Write an algorithm to find the​ latitude​, ​longitude ​and ​confidence ​when the surface
temperature (°C) was between ​65 °C​ and​ 100 °C​. Justify your choice of the data partition
technique and search technique you have used.

**Justification**
For this task we are using hash partition as our partition method and linear search as our searching technique.
The reason behind using hash partitioned.Hence, data is grouped sementically, that is data on the same group shared the same hash value. For range search all proceesors is activated in parallel to do linear search to find the relavant query. 
The reason for using linear search is to avoid pre-sorting as in binary search (alternative way) on all partitions. 
As a result more parallelism happens in this task because all processors are activated.

In [26]:
def parallel_search_range(data, query_range, index_number,n_processor):
    """ Perform parallel search for range selection on data for the given key
    Arguments:
        data -- the input dataset which is a list
        query_range -- a query record in the form of a range (e.g. [30, 50])
        index_number -- index_number of query in the list or record
        n_processor -- the number of parallel processors
    Return:
        results -- the matched record information
    """
    results = []
    # Pool: a Python class enabling parallel processing.
    # Set the number of processors for running at the same time
    pool = Pool(processes = n_processor)
    # Store the partitioned list in DD
    # Each element in DD has a pair (hash key: records)
    DD = hash_partition(data,index_number ,n_processor)
    # Find item in the query range base on the hash which is the key 
    for x in range(query_range[0], query_range[1] + 1):
        hashed_key = x%n_processor
        if hashed_key in DD.keys():
            item = pool.apply(linear_search, (list(DD[hashed_key]), str(x)))
            results.append(item)
    pool.close()
    return results
parallel_search_range(FireData,[65,100],7,3)

[[['-37.642', '149.263', '100', '2017-11-30', '65', '53'],
  ['-37.862', '144.175', '87', '2017-11-14', '65', '88'],
  ['-37.331', '143.122', '90', '2017-11-05', '65', '119'],
  ['-37.46', '148.102', '88', '2017-10-03', '65', '182'],
  ['-37.446', '148.102', '100', '2017-09-24', '65', '213'],
  ['-37.396', '148.086', '100', '2017-09-23', '65', '245'],
  ['-38.3998', '147.064', '89', '2017-08-01', '65', '292'],
  ['-38.4792', '146.3081', '89', '2017-05-22', '65', '420'],
  ['-36.5661', '142.2956', '89', '2017-05-15', '65', '493'],
  ['-35.4666', '142.0749', '89', '2017-05-15', '65', '523'],
  ['-36.7218', '141.6411', '89', '2017-05-11', '65', '608'],
  ['-36.4143', '143.272', '89', '2017-05-04', '65', '925'],
  ['-37.876', '143.7804', '89', '2017-04-19', '65', '1134'],
  ['-36.2618', '141.8783', '89', '2017-04-18', '65', '1196'],
  ['-37.0959', '143.8206', '89', '2017-04-18', '65', '1268'],
  ['-34.9023', '142.0557', '89', '2017-04-18', '65', '1472'],
  ['-36.73', '143.935', '89', '2017

# Task 2.1 (Parallel Join)
Write an algorithm to find ​surface temperature (°C), air temperature (°C), relative
humidity ​ and ​maximum wind speed​. Justify your choice of the data partition technique and join technique you have used.

**Justification**
- For partitioning we used round robin data partitioning to enable load balancing during the broadcasting the one of the tables. Since the aim of parallel processing is to achieve load balance in order to reduce the elapsed time of a job then this data partitioning supports that objective.Additionally, we don't face skew and it is easier to implement.

- We used hash-based join technique in this task becuase the complexity of hash-based join(linear complexity ) is smaller compare to Sort-merge join and Nested-loop join algorithms. Hash-based join is widely accepted as the most efficient join algorithm

- The reason for using divide and broadcast over disjoin Partitioning-Based Parallel Join is to avoid sorting in. From the load balancing point of view, the load of each processor in terms of the number of records processed is the same.However, we do not have problem of workload imbalance because of round robin partitioning. 

In [27]:
def HB_join(T1,T2,index_number,n_processor):
    """Perform the hash-based join algorithm.
    The join attribute is the numeric attribute in the input tables T1 & T2
    
    Arguments:
        T1 & T2 -- Tables to be joined
        index_number -- index_number of query in the list or record
        n_processor -- number of processors
        
    Return:
        result -- the joined table
"""
    result = []
    dic = {}
    # For each record in table T2 do the following:
    for x in T2:
        # Hash the record based on join attribute value using hash function H into hash table
        h = s_hash(x,index_number,n_processor)
        # If h does not exists in keys of dic add it(for the first time of hashing when \
        # a new hash value calculated)
        if (h in dic.keys()):
            s = dic[h] 
            s.append(x)
            dic[h] = s
        # Add x to the relevant hash table
        else:
            s = []
            s.append(x)
            dic[h] = s   
    # For each record in table T1 do the following:
    for r in T1:
        # Hash the record based on join attribute value using hash function H into hash table
        r_key = s_hash(r,6,n_processor)
        # join the the table T1 and T2 base on common attribute
        if r_key in dic:
            for x in dic[r_key]:
                if (x[1] == r[6]):
                    result.append(x+r)
    return result

def DDP_join(T1,T2,index_number,n_processor):
    """ Perform a divide and broadcast-based parallel join algorithms.
    The join attribute is the numeric attribute in the input tables T1 & T2
    
    Arguments:
        T1 & T2 -- Tables to be joined
        n_processor -- the number of parallel processors
        index_number -- index_number of query in the list or record
        
    Return:
        result -- the joined table
    """
    result = []
    results = []
    # Partitioning table T1 to create subsets of T1 
    T1_subsets = rr_partition(T1,n_processor)
    # Pool: a Python class enabling parallel processing.
    # Set the number of processors for running at the same time
    pool = mp.Pool(processes = n_processor)
    # For each record in subsets of T1 do following(Hash Base Join of records of table T1
    # subsets with table T2)
    for t1 in T1_subsets:
        result.append(pool.apply(HB_join, [t1,T2,index_number,n_processor]))
    pool.close()
    # Merging T1 partitions into one list for further data queries
    for item in result:
        for record in item:
            results.append(record)
    return results

Final_merge = DDP_join(FireData,ClimateData,1,3)
for item in Final_merge:   
    print(["Date: " + item[16]+"  surface temperature(°C): "+ item[17] + " --Air Temperature(°C): "+ item[2],
           "--Relative Humidity: "+ item[3],"--Maximum wind Speed: " + item[5]])

['Date: 2017-12-27  surface temperature(°C): 68 --Air Temperature(°C): 28', '--Relative Humidity: 58.3', '--Maximum wind Speed: 15.9']
['Date: 2017-12-27  surface temperature(°C): 67 --Air Temperature(°C): 28', '--Relative Humidity: 58.3', '--Maximum wind Speed: 15.9']
['Date: 2017-12-21  surface temperature(°C): 46 --Air Temperature(°C): 17', '--Relative Humidity: 52.5', '--Maximum wind Speed: 11.1']
['Date: 2017-12-16  surface temperature(°C): 54 --Air Temperature(°C): 18', '--Relative Humidity: 53.7', '--Maximum wind Speed: 13']
['Date: 2017-12-16  surface temperature(°C): 75 --Air Temperature(°C): 18', '--Relative Humidity: 53.7', '--Maximum wind Speed: 13']
['Date: 2017-12-16  surface temperature(°C): 56 --Air Temperature(°C): 18', '--Relative Humidity: 53.7', '--Maximum wind Speed: 13']
['Date: 2017-12-16  surface temperature(°C): 48 --Air Temperature(°C): 18', '--Relative Humidity: 53.7', '--Maximum wind Speed: 13']
['Date: 2017-12-16  surface temperature(°C): 57 --Air Temperatu

# Task 2.2 (Parallel Join)
Write an algorithm to find ​datetime, air temperature (°C), surface temperature (°C)​ and
confidence ​when the ​confidence ​is between 80 and 100. Justify your choice of the data
partition technique and join technique you have used

**Justification**
- For this task we are using hash partition as our partition method and range search as our searching technique.
- Hence, data is grouped sementically, that is data on the same group shared the same hash value. For range search all proceesors is activated in parallel to do range search to find the relavant query. As a result more parallelism happens in this task because all processors are activated.
- The reason behind using range search in this task is because the final results must be of records having the confidence between 80 and 100. This is a filter condition and our code is made flexible with range search such that the filter condition values for confidence can be changed in the search easily because we are passing the values as parameters for filtering.

In [28]:
def parallel_search_range(data, query_range, index_number,n_processor):
    """Perform parallel search for range selection on data for the given key

    Arguments:
        data -- the input dataset which is a list
        query_range -- a query record in the form of a range (e.g. [30, 50])
        index_number: which item (column) in data records with index_number
        n_processor -- the number of parallel processors
            
    Return:
        results -- the matched record information
    """
    
    results = []
    # Pool: a Python class enabling parallel processing.
    pool = Pool(processes = n_processor)
    # Store partitioned data into DD
    DD = h_partition_joined_tables(data,index_number ,n_processor)
    # Do the following for records that are in lower bund and 
    # upper bund of the query range 
    for x in range(query_range[0], query_range[1] + 1):
        # Hashing the x to find the key in partitions
        hashed_key = x%n_processor
        # Find the hashed_key in the partitions
        if hashed_key in DD.keys():
            # Call range search function
            item = pool.apply(range_search,(list(DD[hashed_key]),3,query_range))
            results.append(item)
    pool.close()
    return results
task2_2 = parallel_search_range(Final_merge,[80,100],15,3)
print(task2_2)

[[['2017-12-27T00:02:14', '28', '67', '86'], ['2017-12-16T04:34:58', '18', '75', '95'], ['2017-12-13T04:04:19', '28', '60', '83'], ['2017-11-30T04:34:57', '26', '83', '98'], ['2017-11-30T00:20:53', '26', '58', '83'], ['2017-11-11T13:30:08', '18', '37', '80'], ['2017-11-05T04:41:34', '13', '58', '83'], ['2017-11-05T04:41:19', '13', '62', '89'], ['2017-10-04T04:41:00', '19', '46', '80'], ['2017-10-01T04:10:28', '12', '61', '89'], ['2017-09-27T04:35:36', '15', '56', '83'], ['2017-09-23T23:50:44', '17', '56', '86'], ['2017-09-10T04:00:00', '11', '54', '80'], ['2017-08-13T05:05:10', '15', '53', '80'], ['2017-07-06T04:08:00', '9', '56', '83'], ['2017-05-22T04:32:20', '17', '65', '89'], ['2017-05-22T00:13:00', '17', '64', '89'], ['2017-05-15T04:33:00', '10', '70', '92'], ['2017-05-15T04:30:00', '10', '60', '86'], ['2017-05-15T04:26:20', '10', '53', '80'], ['2017-05-15T04:26:20', '10', '60', '86'], ['2017-05-15T04:26:20', '10', '56', '83'], ['2017-05-13T04:43:30', '9', '77', '95'], ['2017-05-1

# Task 3 (Parallel Sort)
Write an algorithm to sort ​fire data​ based on ​surface temperature​ ​(°C) ​in a ​ascending
order. ​Justify your choice of the data partition technique and sorting technique you have used.

**Justification**
- For partitioning we used round robin data partitioning to enable load balancing. Since the aim of parallel processing is to achieve load balance in order to reduce the elapsed time of a job then this data partitioning supports that objective. Additionally, we don't face skew and it is easier to implement load balancing the processes.

- For internal sorting in one partition we are using serial sorting because if the data does not fit into main memory serial sorting break up the file into unsorted subfiles, sort the subfiles, and then merge the subfiles into larger and larger sorted subfiles until the end.

- The main reason for using parallel binary-merge sort is that the mergin work-load is spread to a pipeline of processors instead of one processor. Additionaly, the advtange is by using the round robin partiton with this method load balancing in the local sort can be done. Finally, after comparison with k-way-merge when it comes to the merging the k-way merging requires a sufficient number of files to be open at the same time since we are using binary merge sort, the binary merging requires a maximum of two files to be opened which can be easily handled compare to k-way-merge.

In [29]:
def qsort(data): 
    """ Quicksort a list
    
    Arguments:
        data -- the input list to be sorted
        
    Return:
        result -- the sorted list
    """
    if len(data) <= 1:
        return data
    else:
        return qsort([x for x in data[1:] if int(x[7]) < int(data[0][7])]) + [data[0]] + \
                qsort([x for x in data[1:] if int(x[7]) >= int(data[0][7])])
        
def serial_sorting(dataset,buffer_size):
    """ Perform a serial external sorting method based on sort-merge
    The buffer size determines the size of eac sub-record set
    
    Arguments:
        dataset -- the entire record set to be sorted
        buffer_size -- the buffer size determining the size of each sub-recordset
        
    Return:
        result -- the sorted record set
    """
    #checking the buffer size
    if buffer_size <= 2 :
        print("buffer size should be greater than two")
        return
    # --- Sort Phase ---
    # Read buffer_size pages at a time into memory and
    # sort them, and write out a sub-record set (i.e. variable: subset)
    sorted_set = []
    start_pos = 0
    N = len(dataset)
    while True:
        if N-start_pos > buffer_size:
            # read B-records from the input, where B = buffer_size
            subset = dataset[start_pos:start_pos+buffer_size]
            # sort the subset (using qucksort defined above)
            sorted_set.append(qsort(subset))
            start_pos = start_pos+buffer_size
        else:
            # read the last B-records from the input, where B is less than buffer_size
            subset = dataset[start_pos:]
            # sort the subset (using qucksort defined above)
            sorted_set.append(qsort(subset))
            break 
            
    # --- Merge Phase ---
    dataset = sorted_set
    merge_buffer_size = buffer_size-1
    while True:
        merged_set = []
        start_pos = 0
        N = len(dataset)
        
        while True:
            if N-start_pos > merge_buffer_size:
                # read C-record sets from the merged record sets, where C= merge_buffer_size
                subset = dataset[start_pos:start_pos+merge_buffer_size]
                item = k_way_merge(subset) # merge lists in subset
                merged_set.append(item)
                start_pos = start_pos + merge_buffer_size
            else:
                # read C-record sets from the merged sets, where C is less 
                # than merge_buffer_size
                subset = dataset[start_pos:]
                item = k_way_merge(subset) # merge lists in subset
                merged_set.append(item)
                break        
        dataset = merged_set
        # If the size of merged record set is 1, then stop        
        if len(dataset) <= 1 :
            result = merged_set
            break 
    return result

def find_min(record):
    """ Find the smallest record
    
    Arguments:
        record -- the input record 
        
    Return:
        result -- the smallest record
    """
    # Checking first index of the record if it is an  integere store it as min
    if type(record[0]) is int:
        minimum = int(record[0])
    # Checking first index of the record if it is an  list store the first item of thath  list
    # as min   
    if type(record[0]) is list:
        minimum = int(record[0][7])
        
    index = 0
    # Finding the minimum of the record
    for i in range(len(record)):
        if type(record[i]) is int:
            if int(record[i]) < minimum:
                minimum = int(record[i])
                index = i
        if type(record[i]) is list: 
            if int(record[i][7]) < minimum:
                minimum = int(record[i][7])
                index = i
    return index

def k_way_merge(record_set):
    """ K-way merging algorithm
    
    Arguments:
        record_sets -- the set of mulitple sorted sub-record sets
    Return:
        result -- the sorted and merged record set
    """
    result = []
    index = []
    # Indexes will keep the indexes of sorted records in the given buffers
    for i in range(len(record_set)):
        index.append(0) # initialisation with 0
    #---- The merging unit ------
    while True:
        tuples = [] # initialise tuple
        # This loop gets the current position of every buffe
        for i in range(len(record_set)):
            if index[i] >= len(record_set[i]):
                tuples.append(sys.maxsize)
            else:
                tuples.append(record_set[i][index[i]])    
        # Find the smallest record
        smallest = find_min(tuples)
        # If we only have sys.maxsize on the tuple, we reached the end of
        # every record set
        if tuples[smallest] == sys.maxsize:
            break
            
        # This record is the next on the merged list    
        result.append(record_set[smallest][index[smallest]])
        index[smallest] += 1
    return result


def parallel_binary_merge_sort(dataset,n_processor,buffer_size):
    """ Perform a parallel binary-merge sorting method

    Arguments:
        dataset -- entire record set to be sorted
        n_processor -- number of parallel processors
        buffer_size -- buffer size determining the size of each sub-record set
        
    Return:
        result -- the merged record set
    """
    result = []
    
    if buffer_size <= 2:
        print("error, buffer size should be greater than 2")
        
    #Perform data partitioning using round-robin partitioning    
    dataset = rr_partition(dataset,n_processor)
    # Pool: a Python class enabling parallel processing.
    # Set the number of processors for running at the same time
    pool = mp.Pool(processes = n_processor)
    # ----- Sort phase -----
    sorted_set = []
    serial_sorted_set = []
    # Creating empty bins for serial_sorted_set
    for i in range(len(dataset)):
        serial_sorted_set.append([])
                   
    for index,item in enumerate(dataset):
        # Call the serial_sorting method above
        serial_sorted_set[index] = pool.apply_async(serial_sorting,[item,buffer_size])    
    pool.close()
    
    # For each item in result set
    for item in serial_sorted_set:
        # get item value
        item = item.get()
        # Unpacks an list or tuple into position arguments
        sorted_set.append(*item)
        
    # ---- Final merge phase ----
    merge_buffer_size = 2 
    dataset = sorted_set
    # Merging the last sets of record two by two 
    while True:
        merged_pool_object = []
        merged_set = []
        start_pos = 0
        N = len(dataset)
        # Pool: a Python class enabling parallel processing.
        # Set the number of processors for running at the same time
        pool = mp.Pool(processes = N//2)
        while True:
        
            if N-start_pos > merge_buffer_size:
                # Read C-record sets from the merged record sets, where C= merge_buffer_size
                subset = dataset[start_pos:start_pos+buffer_size]
                # Call K way merge for subset
                mp_objects = pool.apply_async(k_way_merge,[subset])
                merged_pool_object.append(mp_objects)
                start_pos = start_pos + buffer_size
            
            else:
                # Read C-record sets from the merged sets, where C is less than merge_buffer_size
                subset = dataset[start_pos:]
                # If the subset is not empty  call k way merge for that subset
                if len(subset) != 0:
                    mp_objects = pool.apply_async(k_way_merge,[subset])
                    merged_pool_object.append(mp_objects)
                break
        pool.close()
        # For each item in merged_pool_object
        for item in merged_pool_object:
            # Get item value and append it to merged_set
            item = item.get()
            merged_set.append(item)
            
        dataset = merged_set
        # If the size of dataset is 1, then stop
        if len(dataset) <= 1:
            result = dataset
            break
    
    return result

task3 = parallel_binary_merge_sort(FireData,4,10)
print(task3)

[[['-37.886', '147.207', '302', '2017-07-02T04:28:42', '10.7', '50', '2017-07-02', '28', '312'], ['-37.886', '147.207', '302', '2017-07-02T04:28:42', '10.7', '50', '2017-07-02', '28', '311'], ['-36.943', '143.286', '302.7', '2017-11-11T15:08:00', '18.8', '51', '2017-11-11', '29', '101'], ['-37.062', '141.373', '303.1', '2017-07-01T13:11:41', '16.1', '53', '2017-07-01', '29', '313'], ['-37.466', '148.1', '302.2', '2017-10-02T23:44:31', '10.9', '50', '2017-10-02', '29', '186'], ['-37.062', '141.373', '303.1', '2017-07-01T13:11:41', '16.1', '53', '2017-07-01', '29', '314'], ['-37.227', '141.146', '305.1', '2017-10-03T01:22:44', '41.2', '54', '2017-10-03', '31', '185'], ['-37.38', '149.334', '304.5', '2017-11-30T15:38:32', '14.1', '61', '2017-11-30', '31', '47'], ['-36.779', '146.108', '305.3', '2017-07-01T03:46:08', '25.7', '61', '2017-07-01', '32', '316'], ['-35.646', '142.282', '305.6', '2017-12-24T13:12:01', '11.8', '65', '2017-12-24', '32', '5'], ['-37.598', '149.29', '305.4', '2017-1

# Task 4.1 (Parallel Group-By)
Write an algorithm to get the number of fire in each day. You are required to only display
total number of fire​ ​and ​the date​ in the output. Justify your choice of the data partition
technique if any.

**Justification**
- For partitioning we used round robin data partitioning to enable load balancing in this task. Since the aim of parallel processing is to achieve load balance in order to reduce the elapsed time of a job then this data partitioning supports that objective. Additionally, we don't face skew and it is easier to implement load balancing the processes.
- As we are grouping by based on date of firedata to find the count of fire each day the entire records of firedata must be processed where round robin partition used here divide the records equally among the processors we use and avoid the skew if we use hash partition and the sorting cost issue of range partition instead equal work for all the processors and load balance is obtained.

In [30]:
def local_groupby(dataset,index_number):
    """ Perform a local groupby method
    
    Arguments:
        dataset -- entire record set to be merged
    Return:
        result -- the aggregated record set according to the group_by attribute index
    """
    
    dic = {}
    for item in dataset:
        if item[index_number] not in dic:
            dic[item[index_number]] = 1
        else:            
            dic[item[index_number]] += 1
        
    return dic



def parallel_merge_group_by(dataset,n_processor,index_number):
    """Perform a parallel merge_all groupby method
    
    Arguments:
        dataset -- entire record set to be merged
        n_processor -- number of parallel processors
        index_number -- it is index number of group by column
    Return:
        result -- the number of record dictionary according to the group_by a ttribute index
    """
    
    result = {}
    local_result = []
    # Define the number of parallel processors: the number of sub-datasets
    subsets = rr_partition(dataset, n_processor)
    # Pool: a Python method enabling parallel processing.
    pool = mp.Pool(processes = n_processor)
    
    # ----- Local aggregation step -----
    for s in subsets:
        # call the local aggregation method
        local_result.append(pool.apply(local_groupby,[s,index_number]))
    pool.close()    
    
    # ---- Global aggregation step ----  
    for item in local_result:
        for key in item.keys():
            if key not in result:
                result[key] = item[key]
            else:
                result[key] += item[key]
    return result

parallel_merge_group_by(FireData,4,6)

{'2017-03-06': 2,
 '2017-03-07': 1,
 '2017-03-08': 2,
 '2017-03-09': 3,
 '2017-03-10': 8,
 '2017-03-12': 5,
 '2017-03-13': 2,
 '2017-03-14': 10,
 '2017-03-15': 7,
 '2017-03-17': 6,
 '2017-03-18': 3,
 '2017-03-19': 21,
 '2017-03-24': 2,
 '2017-03-25': 13,
 '2017-03-26': 17,
 '2017-03-28': 54,
 '2017-03-29': 1,
 '2017-03-31': 22,
 '2017-04-01': 7,
 '2017-04-02': 5,
 '2017-04-03': 72,
 '2017-04-04': 89,
 '2017-04-05': 49,
 '2017-04-06': 118,
 '2017-04-07': 39,
 '2017-04-08': 20,
 '2017-04-11': 24,
 '2017-04-12': 69,
 '2017-04-13': 357,
 '2017-04-14': 18,
 '2017-04-15': 69,
 '2017-04-16': 18,
 '2017-04-17': 38,
 '2017-04-18': 325,
 '2017-04-19': 50,
 '2017-04-20': 31,
 '2017-04-22': 2,
 '2017-04-23': 19,
 '2017-04-24': 8,
 '2017-04-25': 3,
 '2017-04-26': 1,
 '2017-04-29': 3,
 '2017-05-01': 20,
 '2017-05-02': 10,
 '2017-05-03': 64,
 '2017-05-04': 135,
 '2017-05-05': 31,
 '2017-05-06': 17,
 '2017-05-07': 3,
 '2017-05-08': 24,
 '2017-05-09': 13,
 '2017-05-10': 114,
 '2017-05-11': 19,
 '2017-0

# Task 4.2 (Parallel Group by)
Write an algorithm to find the ​average surface temperature​ ​(°C) ​for each day​.​ You are
required to only display ​average surface temperature (°C)​ ​and ​the date​ in the output.
Justify your choice of the data partition technique if any

**Justification**
For partitioning we used round robin data partitioning to enable load balancing in this task. Since the aim of parallel processing is to achieve load balance in order to reduce the elapsed time of a job then this data partitioning supports that objective. Additionally, we don't face skew and it is easier to implement load balancing the processes.
As we are grouping by based on date of firedata to find the average surface temperature the entire records of firedata must be processed where round robin partition used here divide the records equally among the processors we use and avoid the skew if we use hash partition and the sorting cost issue of range partition instead equal work for all the processors and load balance is obtained.

In [31]:
def local_groupby_avg(dataset,index_number1,index_number2):
    """ Perform a local groupby method
    
    Arguments:
        dataset -- entire record set to be merged
        index_number1 -- it is index number of group by column
        index_number2 -- the value we want to calculate average
    Return:
        result -- the aggregated record set according to the group_by attribute index
    """
    
    dic = {}
    # Adding the item[7] as a key to dic
    for item in dataset:
        # Adding the not existed key= item[index_number] to the dic
        if item[index_number1] not in dic:
            my_list = []
            my_list.append(int(item[index_number2]))
            dic.update({item[index_number1]: my_list})
        # Adding the value of item to existed dic keys    
        else:  
            item_list = dic[item[index_number1]]  
            item_list.append(int(item[index_number2]))
            dic[item[index_number1]] = item_list
            
    result={}
    # Storing the sum and length of the group by query
    for x in dic.keys():
        averages = []
        sum_record = sum(dic[x])
        averages.append(sum_record)
        len_record = len(dic[x])
        averages.append(len_record)
        result[x] = averages
    return result
    


def parallel_merge_group_by_avg(dataset,n_processor,index_number1,index_number2):
    """Perform a parallel merge_all groupby method and finding average
    
    Arguments:
        dataset -- entire record set to be merged
        n_processor -- number of processor that we want to assign for parallelism
        index_number1 -- it is index number of group by column
        index_number2 -- the value we want to calculate average
    Return:
        result -- the average record dictionary according to the group_by a ttribute index
    """
    result = {}
    local_result = []
    # Define the number of parallel processors: the number of sub-datasets
    subsets = rr_partition(dataset, n_processor)
    # Pool: a Python method enabling parallel processing.
    pool = mp.Pool(processes = n_processor)
    
    # Call local_groupby_avg 
    for s in subsets:
        local_result.append(pool.apply(local_groupby_avg,[s,index_number1,index_number2]))
    pool.close()    
    # Merge local_result sets into one with the same indexes
    for item in local_result:
        for key in item.keys():
            if key not in result:
                result[key] = item[key]
            else:
                result[key][0] += item[key][0]
                result[key][1] += item[key][1]
                
    final_averages = {}
    for x in result.keys():
        final_averages[x] = result[x][0]/result[x][1]
    return final_averages

parallel_merge_group_by_avg(FireData,3,6,7)

{'2017-03-06': 60.5,
 '2017-03-07': 64.0,
 '2017-03-08': 51.5,
 '2017-03-09': 46.666666666666664,
 '2017-03-10': 69.375,
 '2017-03-12': 88.2,
 '2017-03-13': 38.5,
 '2017-03-14': 65.6,
 '2017-03-15': 46.0,
 '2017-03-17': 59.5,
 '2017-03-18': 79.33333333333333,
 '2017-03-19': 65.57142857142857,
 '2017-03-24': 49.0,
 '2017-03-25': 66.0,
 '2017-03-26': 56.88235294117647,
 '2017-03-28': 60.925925925925924,
 '2017-03-29': 51.0,
 '2017-03-31': 48.72727272727273,
 '2017-04-01': 46.714285714285715,
 '2017-04-02': 45.2,
 '2017-04-03': 58.44444444444444,
 '2017-04-04': 62.57303370786517,
 '2017-04-05': 53.142857142857146,
 '2017-04-06': 61.71186440677966,
 '2017-04-07': 50.69230769230769,
 '2017-04-08': 60.75,
 '2017-04-11': 46.291666666666664,
 '2017-04-12': 52.69565217391305,
 '2017-04-13': 58.57983193277311,
 '2017-04-14': 61.94444444444444,
 '2017-04-15': 59.57971014492754,
 '2017-04-16': 48.72222222222222,
 '2017-04-17': 50.921052631578945,
 '2017-04-18': 53.36615384615384,
 '2017-04-19': 54

# Task 5 (Parallel Group by join)
Write an algorithm to find the ​average surface temperature​ ​(°C)​ for each weather station.
You are required to only display ​average surface temperature (°C)​ ​and ​the station ​in the
output.​ ​Justify your choice of the data partition and join technique.

**Justification**
- For partitioning we used round robin data partitioning to enable load balancing in this task. Since the aim of parallel processing is to achieve load balance in order to reduce the elapsed time of a job then this data partitioning supports that objective. Additionally, we don't face skew and it is easier to implement load balancing the processes.
- We used hash-based join technique in this task becuase the complexity of hash-based join(linear complexity ) is smaller compare to Sort-merge join and Nested-loop join algorithms. Hash-based join is widely accepted as the most efficient join algorithm.
- The reason for using divide and broadcast over disjoin Partitioning-Based Parallel Join is to avoid sorting. From the load balancing point of view, the load of each processor in terms of the number of records processed is the same.However, we do not have problem of workload imbalance because of round robin partitioning. 
- As we are grouping by based on station of ClimateData to find the average surface temperature from the FireData the entire records of firedata must be processed where round robin partition used here divide the records equally among the processors we use and avoid the skew if we use hash partition and the sorting cost issue of range partition instead equal work for all the processors and load balance is obtained.

**Parallel Steps**
1. Data partitioning
2. Join operation in parallel
3. Aggregate operation for each processor
4. Consolidation

In [32]:
def HB_join_groupby(T1,T2,index_number,n_processor):
    """Perform the hash-based join algorithm.
    The join attribute is the numeric attribute in the input tables T1 & T2
    
    Arguments:
        T1 & T2 -- Tables to be joined
        index_number -- index_number of query in the list or record
        n_processor -- number of processors
        
    Return:
        result -- the joined table
"""
    result = []
    dic = {}
    # For each record in table T2 do the following:
    for x in T2:
        # Hash the record based on join attribute value using hash function H into hash table
        h = s_hash(x,index_number,n_processor)
        # If h does not exists in keys of dic add it(for the first time of hashing when \
        # a new hash value calculated)
        if (h in dic.keys()):
            s = dic[h] 
            s.append(x)
            dic[h] = s
        # Add x to the relevant hash table
        else:
            s = []
            s.append(x)
            dic[h] = s
            
    # For each record in table T1 do the following:
    for r in T1:
        #print(r)
        # Hash the record based on join attribute value using hash function H into hash table
        r_key = s_hash(r,1,n_processor)
        # join the the table T1 and T2 base on common attribute
        if r_key in dic:
            for x in dic[r_key]:
                if (x[6] == r[1]):
                    result.append(x+r)
        
    return result

def DDP_join(T1,T2,index_number,n_processor):
    """ Perform a divide and broadcast-based parallel join algorithms.
    The join attribute is the numeric attribute in the input tables T1 & T2
    
    Arguments:
        T1 & T2 -- Tables to be joined
        n_processor -- the number of parallel processors
        index_number -- index_number of query in the list or record
        
    Return:
        result -- the joined table
    """
    result = []
    results = {}
    #----step 1: Data Partitioning ---------
    # Partitioning table T1 to create subsets of T1 
    T1_subsets = rr_partition(T1,n_processor)
    # Pool: a Python class enabling parallel processing.
    # Set the number of processors for running at the same time
    pool = mp.Pool(processes = n_processor)
    #-------step2: join operation ----------
    # For each record in subsets of T1 do following(Hash Base Join of records of table T1
    # subsets with table T2)
    for t1 in T1_subsets:
        result.append(pool.apply(HB_join_groupby, [t1,T2,index_number,n_processor]))
    
    #------ step3 : Aggregation
    local_result = []
    # Call local_groupby_avg
    for item in result:
        local_result.append(local_groupby_avg(item,9,7))
    #-------Consolidation-----
    # adding the sum of record and length of record to each other
    for item in local_result:
        for key in item.keys():
            if key not in results:
                results[key] = item[key]
            else:
                results[key][0] += item[key][0]
                results[key][1] += item[key][1]
    
    final_averages = {}
    # Calculating the average
    for item in results:
        final_averages[item] = results[item][0]/results[item][1]
    
    pool.close()
    return final_averages
DDP_join(ClimateData,FireData,6,3)

{'948701': 56.06938603868797, '948702': 52.148275862068964}